# CORD

## CORD Library

In [19]:
%load_ext autoreload
%autoreload 2
from cord.cord19 import *
from cord.core import parallel, ifnone
pd.options.display.max_colwidth = 200

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. Reading the Research Papers

## From Paths

In [20]:
%time research_papers = ResearchPapers.from_data_dir()

Loading metadata from data\CORD-19-research-challenge
Cleaning metadata
Fixing dates that are a list e.g. ['2020-02-05', '2020-02']
Fixing dates with the seasons e.g. 2014 Autumn
Fix dates like 2016 Nov 9 Jan-Feb
Fix dates like 2012 Jan-Mar
Convert Dates like 2020 Apr 13
Converting Dates like 2020 Apr
Converting Dates like 2020
Converting Dates like 2020-01-21
Indexing research papers
Finished Indexing in 55.0 seconds
Wall time: 1min 5s


In [21]:
research_papers.save()

Saving to data\ResearchPapers.pickle


## From Pickle

In [4]:
%time research_papers = ResearchPapers.from_pickle()

## Search Bar

In [22]:
research_papers.searchbar('vaccine transmission')

interactive(children=(Text(value='vaccine transmission', description='SearchTerms'), Output()), _dom_classes=(…

## Search

In [17]:
research_papers.search('hospital doctors', 
                       start_date='2020-01-30', 
                       end_date='2020-02-15',
                       covid_related=True)

title,authors,when,Score
A rapid advice guideline for the diagnosis and treatment of 2019 novel coronavirus (2019-nCoV) infected pneumonia (standard version),"Jin, Ying-Hui; Cai, Lin; Cheng, Zhen-Shun; Cheng, Hong; Deng, Tong; Fan, Yi-Pin; Fang, Cheng; Huang, Di; Huang, Lu-Qi; Huang, Qiao; Han, Yong; Hu, Bo; Hu, Fen; Li, Bing-Hui; Li, Yi-Rong; Liang, Ke; Lin, Li-Kai; Luo, Li-Sha; Ma, Jing; Ma, Lin-Lu; Peng, Zhi-Yong; Pan, Yun-Bao; Pan, Zhen-Yu; Ren, Xue-Qun; Sun, Hui-Min; Wang, Ying; Wang, Yun-Yun; Weng, Hong; Wei, Chao-Jie; Wu, Dong-Fang; Xia, Jian; Xiong, Yong; Xu, Hai-Bo; Yao, Xiao-Mei; Yuan, Yu-Feng; Ye, Tai-Sheng; Zhang, Xiao-Chun; Zhang, Ying-Wen; Zhang, Yin-Gao; Zhang, Hua-Min; Zhao, Yan; Zhao, Ming-Juan; Zi, Hao; Zeng, Xian-Tao; Wang, Yong-Yan; Wang, Xing-Huan",1 month ago,7.5
Diagnosis and treatment recommendations for pediatric respiratory infection caused by the 2019 novel coronavirus,"Chen, Zhi-Min; Fu, Jun-Fen; Shu, Qiang; Chen, Ying-Hu; Hua, Chun-Zhen; Li, Fu-Bang; Lin, Ru; Tang, Lan-Fang; Wang, Tian-Lin; Wang, Wei; Wang, Ying-Shuo; Xu, Wei-Ze; Yang, Zi-Hao; Ye, Sheng; Yuan, Tian-Ming; Zhang, Chen-Mei; Zhang, Yuan-Yuan",1 month ago,3.5
"Epidemiological and Clinical Characteristics of 17 Hospitalized Patients with 2019 Novel Coronavirus Infections Outside Wuhan, China",Jie Li; Shilin Li; Yurui Cai; Qin Liu; Xue Li; Zhaoping Zeng; Yanpeng Chu; Fangcheng Zhu; Fanxin Zeng,1 month ago,3.3
Epidemiologic characteristics of early cases with 2019 novel coronavirus (2019-nCoV) disease in Republic of Korea,"Ki, Moran; nCoV, Task Force For",1 month ago,3.3
Neutrophil-to-Lymphocyte Ratio Predicts Severe Illness Patients with 2019 Novel Coronavirus in the Early Stage,Jingyuan Liu; Yao Liu; Pan Xiang; Lin Pu; Haofeng Xiong; Chuansheng Li; Ming Zhang; Jianbo Tan; Yanli Xu; Rui Song; Meihua Song; Lin Wang; Wei Zhang; Bing Han; Li Yang; Xiaojing Wang; Guiqin Zhou; Ting Zhang; Ben Li; Yanbin Wang; Zhihai Chen; Xianbo Wang,1 month ago,2.9
Statistical Inference for Coronavirus Infected Patients in Wuhan,Yongdao Zhou; Jianghu Dong,1 month ago,2.6
Long-Term Persistence of IgG Antibodies in SARS-CoV Infected Healthcare Workers,Xiaoqin Guo; Zhongmin Guo; Chaohui Duan; Zeliang chen; Guoling Wang; Yi Lu; Mengfeng Li; Jiahai Lu,1 month ago,2.5
Clinical diagnosis of 8274 samples with 2019-novel coronavirus in Wuhan,Ming Wang; Qing Wu; Wanzhou Xu; Bin Qiao; Jingwei Wang; Hongyun Zheng; Shupeng Jiang; Junchi Mei; Zegang Wu; Yayun Deng; Fangyuan Zhou; Wei Wu; Yan Zhang; Zhihua Lv; Jingtao Huang; Xiaoqian Guo; Lina Feng; Zunen Xia; Di Li; Zhiliang Xu; Tiangang Liu; Pingan Zhang; Yongqing Tong; Yan Li,1 month ago,2.4
"Complete genome characterisation of a novel coronavirus associated with severe human respiratory disease in Wuhan, China","Wu, F.; Zhao, S.; Yu, B.; Chen, Y.-M.; Wang, W.; Hu, Y.; Song, Z.-G.; Tao, Z.-W.; Tian, J.-H.; Pei, Y.-Y.; Yuan, M.-L.; Zhang, Y.-L.; Dai, F.-H.; Liu, Y.; Wang, Q.-M.; Zheng, J.-J.; Xu, L.; Holmes, E. C.; Zhang, Y.-Z.",1 month ago,2.4
Clinical analysis of 10 neonates born to mothers with 2019-nCoV pneumonia,"Zhu, Huaping; Wang, Lin; Fang, Chengzhi; Peng, Sicong; Zhang, Lianhong; Chang, Guiping; Xia, Shiwen; Zhou, Wenhao",1 month ago,2.3


In [18]:
research_papers.search('remdesivir')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dwight\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


title,authors,when,Score
Remdesivir as a possible therapeutic option for the COVID-19,"Al-Tawfiq, Jaffar A.; Al-Homoud, Ali H.; Memish, Ziad A.",3 weeks ago,12.7
Arguments in favour of remdesivir for treating SARS-CoV-2 infections,"Ko, Wen-Chien; Rolain, Jean-Marc; Lee, Nan-Yao; Chen, Po-Lin; Huang, Ching-Tai; Lee, Ping-Ing; Hsueh, Po-Ren",2 weeks ago,12.6
Strategies against the novel coronavirus: Possible applications of the experimental Ebola drug remdesivir are being tested,nan,2 months ago,12.1
Remdesivir and chloroquine effectively inhibit the recently emerged novel coronavirus (2019-nCoV) in vitro,"Wang, Manli; Cao, Ruiyuan; Zhang, Leike; Yang, Xinglou; Liu, Jia; Xu, Mingyue; Shi, Zhengli; Hu, Zhihong; Zhong, Wu; Xiao, Gengfu",2 months ago,12.1
Discovering drugs to treat coronavirus disease 2019 (COVID-19),"Dong, Liying; Hu, Shasha; Gao, Jianjun",3 weeks ago,9.0
Potential antiviral therapeutics for 2019 Novel Coronavirus,"Li, H.; Wang, Y. M.; Xu, J. Y.; Cao, B.",2 months ago,7.8
Teicoplanin: an alternative drug for the treatment of coronavirus COVID-19?,"Baron, Sophie Alexandra; Devaux, Christian; Colson, Philippe; Raoult, Didier; Rolain, Jean-Marc",1 week ago,7.7
"Comparative therapeutic efficacy of remdesivir and combination lopinavir, ritonavir, and interferon beta against MERS-CoV","Sheahan, Timothy P.; Sims, Amy C.; Leist, Sarah R.; Schäfer, Alexandra; Won, John; Brown, Ariane J.; Montgomery, Stephanie A.; Hogg, Alison; Babusis, Darius; Clarke, Michael O.; Spahn, Jamie E.; Bauer, Laura; Sellers, Scott; Porter, Danielle; Feng, Joy Y.; Cihlar, Tomas; Jordan, Robert; Denison, Mark R.; Baric, Ralph S.",2 months ago,7.6
Gilead and Moderna lead on coronavirus treatments,nan,2 weeks ago,7.5
Remdesivir (GS-5734) protects African green monkeys from Nipah virus challenge,"Lo, Michael K.; Feldmann, Friederike; Gary, Joy M.; Jordan, Robert; Bannister, Roy; Cronin, Jacqueline; Patel, Nishi R.; Klena, John D.; Nichol, Stuart T.; Cihlar, Tomas; Zaki, Sherif R.; Feldmann, Heinz; Spiropoulou, Christina F.; de Wit, Emmie",,7.5
